# 🧠 Gym Progress Analysis – User: Kuba

---

## 📋 Overview

This notebook presents an exploratory data analysis (EDA) of gym training data for a single user: **Kuba**.  
The goal is to understand his training patterns, volume progression, performance changes over time, and relationships between training variables (e.g. sleep, mood, stress).

We will use data loaded from an SQLite database that contains structured records of:

- Training sessions (date, bodyweight, mood, measurements, etc.)
- Exercises (name, muscle group, weight, reps)
- Additional context (program phase, duration, stress level)

---

## 🎯 Objectives

- Visualize training frequency and exercise history
- Track progression of specific lifts (e.g. Bench Press, Deadlift)
- Analyze weekly training volume per muscle group
- Examine correlations between training variables
- Prepare data for future modeling (e.g. predicting strength or body changes)

---


In [1]:
# 1️⃣ Wczytaj zmienne środowiskowe (.env)
from dotenv import load_dotenv
load_dotenv()  

# 2️⃣ Import zapytań – teraz database.queries jest dostępne
from database.queries import get_all_users, get_training_data_for_user

# 3️⃣ Uruchomienie zapytań
users = get_all_users()
print("Users:", users)

rows = get_training_data_for_user(1)
print(f"Records for user 1 ({len(rows)} rows):")
for r in rows[:5]:
    print(r)


ModuleNotFoundError: No module named 'dotenv'

In [ ]:
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, project_root)

from database.queries import get_all_users, get_training_data_for_user

users = get_all_users()
print("Users:", users)

rows = get_training_data_for_user(1)
print(f"Records for user 1 ({len(rows)} rows):")
for r in rows[:5]:
    print(r)


In [10]:
from database.queries import get_all_users, get_training_data_for_user

users = get_all_users()
print("Users:", users)

rows = get_training_data_for_user(1)
print(f"Records for user 1 ({len(rows)} rows):")
for r in rows[:5]:
    print(r)


OperationalError: unable to open database file